<a href="https://colab.research.google.com/github/Utkarshp1/Bayesian_Optimisation/blob/master/Building_Blocks_of_Ax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Blocks of Ax

In [ ]:
!pip3 install ax-platform

     |████████████████████████████████| 819kB 13.0MB/s 
     |████████████████████████████████| 481kB 33.5MB/s 
     |████████████████████████████████| 501kB 39.6MB/s 


This tutorial illustrates the core Ax classes and their usage by constructing, running, and saving an experiment through the Developer API.

In [ ]:
import pandas as pd
from ax import *

## 1. Define the Search Space

Here we range over two parameters, each of which can take on values between 0 and 10.

In [ ]:
range_param1 = RangeParameter(name='x1', lower=0.0, upper=10.0, parameter_type=ParameterType.FLOAT)
range_param2 = RangeParameter(name='x2', lower=0.0, upper=10.0, parameter_type=ParameterType.FLOAT)

search_space = SearchSpace(
    parameters=[range_param1, range_param2]
)

## 2. Define the experiment 

In [ ]:
experiment = Experiment(
    name='experiment_building_blocks',
    search_space=search_space
)

## 3. Generate Arms

We can now generate arms, i.e. assignments of parameters to values, that lie within the search space. Below we use a Sobol generator to generate five quasi-random arms. The `Models` registry provides a set of standard models Ax contains.

In [ ]:
sobol = Models.SOBOL(search_space=experiment.search_space)
generator_run = sobol.gen(5)

for arm in generator_run.arms:
    print(arm)

Arm(parameters={'x1': 5.87591290473938, 'x2': 0.7750853896141052})
Arm(parameters={'x1': 1.9139529298990965, 'x2': 5.136057734489441})
Arm(parameters={'x1': 3.9543073624372482, 'x2': 4.898538440465927})
Arm(parameters={'x1': 7.643105583265424, 'x2': 9.288824377581477})
Arm(parameters={'x1': 9.57068582996726, 'x2': 3.117052325978875})


## 4. Define an optimization config with custom metrics

In order to perform an optimization, we also need to define an optimization config for the experiment. An optimization config is composed of an objective metric to be minimized or maximized in the experiment, and optionally a set of outcome constraints that place restrictions on how other metrics can be moved by the experiment.

In order to define an objective or outcome constraints, we first need to subclass `Metric`. Metrics are used to evaluate trials, which are individual steps of the experiment sequence. Each trial contains one or more arms for which we will collect data at the same time.

Our custom metric(s) will determine how, given a trial, to compute the mean and SEM of each of the trial's arms.

**TERMINOLOGIES:**
* **Metric:** Interface for fetching data for a specific measurement on an experiment or trial.
* **Objective:** The metric to be optimized, with an optimization direction (maximize/minimize).
* **SEM:** Standard error fof the metric's mean, 0.0 for noiseless measurements. If no value is provided, defaults to `np.nan`, in which case Ax infers its value using the measurements collected during experimentation.

In [ ]:
class BoothMetric(Metric):
    def fetch_trial_data(self, trial):
        records=[]
        for arm_name, arm in trial.arms_by_name.items():
            params = arm.parameters
            records.append({
                "arm_name": arm_name,
                "metric_name": self.name,
                "mean": (params["x1"] + 2*params["x2"] - 7)**2 + (2*params["x1"] + params["x2"] - 5)**2,
                "sem": 0.0,
                "trial_index": trial.index,
            })
        return Data(df=pd.DataFrame.from_records(records))

In [ ]:
optimization_config = OptimizationConfig(
    objective = Objective(
        metric=BoothMetric(name="booth"),
        minimize=True
    ),
)

experiment.optimization_config = optimization_config

## 5. Define a runner

Before an experiment can collect data, it must have a `Runner` attached. A runner handles the deployment of trials. A trial must be "run" before it can be evaluated.

Here, we have a dummy runner that does nothing. In practice, a runner might be in charge of pushing an experiment to production.

In [ ]:
class MyRunner(Runner):
    def run(self, trial):
        return {"name": str(trial.index)}

experiment.runner = MyRunner()

## 6. Create a trial

Now we can collect data for arms within our search space and begin the optimization. We do this by:
1. Generating arms for an initial exploratory batch (already done above, using Sobol)
2. Adding these arms to trial
3. Running the trial
4. Evaluating the trial
5. Generating new arms based on the results, and repeating

In [ ]:
experiment.new_batch_trial(generator_run=generator_run)

BatchTrial(experiment_name='experiment_building_blocks', index=0, status=TrialStatus.CANDIDATE)

Note that the arms attached to the trial are the same as those in the generator run above, except for the status quo, which is automatically added to each trial. This can be confirmed by using the following:

In [ ]:
for arm in experiment.trials[0].arms:
    print(arm)

Arm(name='0_0', parameters={'x1': 5.87591290473938, 'x2': 0.7750853896141052})
Arm(name='0_1', parameters={'x1': 1.9139529298990965, 'x2': 5.136057734489441})
Arm(name='0_2', parameters={'x1': 3.9543073624372482, 'x2': 4.898538440465927})
Arm(name='0_3', parameters={'x1': 7.643105583265424, 'x2': 9.288824377581477})
Arm(name='0_4', parameters={'x1': 9.57068582996726, 'x2': 3.117052325978875})


Check the values of the parameters in `generator_run` defined above and here are the same.

If our trial should contain only one arm, we can use `experiment.new_trial` instead.

```python
    experiment.new_trial().add_arm(Arm(name='single_arm', parameters={'x1': 1, 'x2': 1}))
```

The arm then can be checked as follows:

```python
    print(experiment.trials[1].arm)
```

## 7. Fetch Data

To fetch trial data, we need to run it and mark it completed. For most metrics in Ax, data is only available once the status of the trial is `COMPLETED`, since in real-world scenarios, metrics can typically only be fetched after the trial finished running.

NOTE: Metrics classes may implement the `is_available_while_running` method. When this method returns `True` data is available when trials are either `RUNNING` or `COMPLETED`. This can be used to obtain intermediate results from A/B test trials and other online experiments, or when metric values are available immediately, like in the case of synthetic problem metrics.

In [ ]:
experiment.trials[0].run().mark_completed()

BatchTrial(experiment_name='experiment_building_blocks', index=0, status=TrialStatus.COMPLETED)

In [ ]:
data = experiment.fetch_data()
data.df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,booth,56.835940,0.0,0
1,0_1,booth,42.608313,0.0,0
2,0_2,booth,106.532830,0.0,0
3,0_3,booth,752.619414,0.0,0
4,0_4,booth,375.377534,0.0,0


## 8. Iterate using GP+EI

Now we can model the data collected for the initial set of arms via Bayesian Optimization (using the BoTorch model default of Gaussian Process with Expected Improvement acquistion function) to determine the new arms for which to fetch data next.

In [ ]:
gpei = Models.BOTORCH(experiment=experiment, data=data)
generator_run = gpei.gen(5)
experiment.new_batch_trial(generator_run=generator_run)

BatchTrial(experiment_name='experiment_building_blocks', index=1, status=TrialStatus.CANDIDATE)

In [ ]:
for arm in experiment.trials[1].arms:
    print(arm)

Arm(name='1_0', parameters={'x1': 2.065329696796488, 'x2': 1.969113723313942})
Arm(name='1_1', parameters={'x1': 0.0, 'x2': 7.654470729074037})
Arm(name='1_2', parameters={'x1': 0.0, 'x2': 0.0})
Arm(name='1_3', parameters={'x1': 0.0, 'x2': 3.3270942424052077})
Arm(name='1_4', parameters={'x1': 3.234594058879759, 'x2': 0.0})


In [ ]:
experiment.trials[1].run().mark_completed()
data = experiment.fetch_data()
data.df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,booth,56.835940,0.0,0
1,0_1,booth,42.608313,0.0,0
2,0_2,booth,106.532830,0.0,0
3,0_3,booth,752.619414,0.0,0
4,0_4,booth,375.377534,0.0,0
5,1_0,booth,2.202399,0.0,1
6,1_1,booth,76.084723,0.0,1
7,1_2,booth,74.000000,0.0,1
8,1_3,booth,2.918199,0.0,1
9,1_4,booth,16.336796,0.0,1


In [ ]:
data = experiment.fetch_trials_data(trial_indices=range(1, 2))
data.df

,arm_name,metric_name,mean,sem,trial_index
0,1_0,booth,2.202399,0.0,1
1,1_1,booth,76.084723,0.0,1
2,1_2,booth,74.000000,0.0,1
3,1_3,booth,2.918199,0.0,1
4,1_4,booth,16.336796,0.0,1
